In [1]:
from checkreq import FindImport, FindreqLibraries, changereqfile

['toml @ file:///tmp/build/80754af9/toml_1616166611790/work', 'tomli @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/tomli_1699237289925/work', 'tomlkit @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/tomlkit_1699238737474/work']


In [2]:
import os

def GetFilesInDirectory(directory):
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            paths.append(file_path)
    # print(paths)
    return paths


In [3]:
list_of_libs = []

for dir in GetFilesInDirectory('src'):
    list_of_libs += FindImport(dir)

In [4]:
list_of_libs

['langchain',
 'json',
 'boto3',
 'datetime',
 'asyncio',
 'base64',
 'requests',
 'time']

In [5]:
for dir in GetFilesInDirectory('src'):
    changereqfile("requirements2.txt", 'requirements.txt', dir)
    print(FindreqLibraries("requirements.txt", dir))

['langchain==0.1.16', 'langchain-aws==0.1.2', 'langchain-cohere==0.1.5', 'langchain-community==0.0.34', 'langchain-core==0.1.46', 'langchain-openai==0.0.2.post1', 'langchain-text-splitters==0.0.1', 'json5 @ file:///tmp/build/80754af9/json5_1624432770122/work', 'jsonpatch @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_3ajyoz8zoj/croot/jsonpatch_1714483362270/work', 'jsonpointer==2.1', 'jsonref==1.1.0', 'jsonschema @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema_1701803145350/work', 'jsonschema-specifications @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema-specifications_1701803122948/work', 'boto3==1.34.84']
['requests @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b143dxenr7/croot/requests_1716902835977/work', 'requests-file @ file:///Users/ktietz/demo/mc3/conda-bld/requests-file_1629455781986/work', 'requests-oauthlib==1.3.1', 'requests-toolbelt @ file:///Users/builder/cbouss/persev

In [7]:
import dspy
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
import google.generativeai as genai
from dspy import settings


c:\Users\mitul\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:


gemini = dspy.Google("models/gemini-1.5-flash",
                         api_key="YOUR-API-KEY")



# model_kwargs =  { 
#         "max_tokens": 4096,
#         "temperature": 0.4,
#         "top_k": 50,
#         "top_p": 0.9
#     }

# lm = dspy.GeminiModel(gemini, "gemini.model_name_version", **model_kwargs)

In [19]:
# dspy.configure(lm=lm)
dspy.settings.configure(lm=gemini)

In [4]:
## Get input fields through LLM again

In [20]:
class Input(BaseModel):
    python_version: str = Field(description="Python version to be used")
    type_of_application: str = Field(description="Type of python application")
    work_directory: str = Field(description="Working directory of the application")
    entrypoint_filename: str = Field(description="File name for starting the application endpoint")

class Output(BaseModel):
    dockerfile: str = Field(description="The complete dockerfile code")


class CreateDockerFile(dspy.Signature):
    """I need a Dockerfile for a basic Streamlit app. The app is written in Python."""

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()


In [21]:
predictor = dspy.TypedPredictor(CreateDockerFile)
input = Input(
    python_version = "3.11",
    type_of_application = 'Streamlit',
    work_directory = 'src',
    entrypoint_filename = 'app.py'
)
mapped_parameters = predictor(input=input)

In [22]:
print(mapped_parameters.output.dockerfile)

FROM python:${python_version}

WORKDIR /app

COPY ${work_directory} .

RUN pip install -r requirements.txt

CMD ["streamlit", "run", "${entrypoint_filename}"]


In [15]:
with open("Dockerfile", "w") as file:
    file.write(mapped_parameters.output.dockerfile)